### Challenge 29: Break a SHA-1 keyed MAC using length extension

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

<div class="alert alert-block alert-info">

Secret-prefix SHA-1 MACs are trivially breakable.

The attack on secret-prefix SHA1 relies on the fact that you can take the ouput of SHA-1 and use it as a new starting point for SHA-1, thus taking an arbitrary SHA-1 hash and "feeding it more data".

Since the key precedes the data in secret-prefix, any additional data you feed the SHA-1 hash in this fashion will appear to have been hashed with the secret key.

To carry out the attack, you'll need to account for the fact that SHA-1 is "padded" with the bit-length of the message; your forged message will need to include that padding. We call this "glue padding". The final message you actually forge will be:

```SHA1(key || original-message || glue-padding || new-message)```

(where the final padding on the whole constructed message is implied)

Note that to generate the glue padding, you'll need to know the original bit length of the message; the message itself is known to the attacker, but the secret key isn't, so you'll need to guess at it.

This sounds more complicated than it is in practice.

To implement the attack, first write the function that computes the MD padding of an arbitrary message and verify that you're generating the same padding that your SHA-1 implementation is using. This should take you 5-10 minutes.

Now, take the SHA-1 secret-prefix MAC of the message you want to forge --- this is just a SHA-1 hash --- and break it into 32 bit SHA-1 registers (SHA-1 calls them "a", "b", "c", &c).

Modify your SHA-1 implementation so that callers can pass in new values for "a", "b", "c" &c (they normally start at magic numbers). With the registers "fixated", hash the additional data you want to forge.

Using this attack, generate a secret-prefix MAC under a secret key (choose a random word from /usr/share/dict/words or something) of the string:

```"comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon"```

Forge a variant of this message that ends with ```";admin=true"```.

<div class="alert alert-block alert-warning">

This is a very useful attack.

For instance: Thai Duong and Juliano Rizzo, who got to this attack before we did, used it to break the Flickr API.

</div>


</div>

Found this implementation from [https://gist.github.com/BenWiederhake/cb60f703840f9e81a84499b39eb361b5](https://gist.github.com/BenWiederhake/cb60f703840f9e81a84499b39eb361b5)

In [1]:
from Crypto import Random
import sha1

<div class="alert alert-block alert-info">
    
To implement the attack, first write the function that computes the MD padding of an arbitrary message and verify that you're generating the same padding that your SHA-1 implementation is using. This should take you 5-10 minutes.
    
</div>

From [Wikipedia pseudocode](https://en.wikipedia.org/wiki/SHA-1), 

ml = message length in bits (always a multiple of the number of bits in a character).

Pre-processing:

- append the bit '1' to the message e.g. by adding 0x80 if message length is a multiple of 8 bits.
- append 0 ≤ k < 512 bits '0', such that the resulting message length in bits is congruent to −64 ≡ 448 (mod 512)
- append ml, the original message length, as a 64-bit big-endian integer. 

Thus, the total length is a multiple of 512 bits.

In [2]:
def compute_sha1_padding(data):
    
    # Let's assume message / data is always complete bytes, no extra bits, so always append 0x80
    msg_len = len(data)
    
    # Pad with 0’s until message is 64-bits less than some multiple of 512 (64 in bytes)
    n_zero_bytes = ( (56 - ( (msg_len % 64) + 1) ) % 64 ) 
    
    padding = b'\x80' + b'\x00'*n_zero_bytes + (msg_len*8).to_bytes(8, 'big')
    return(padding)

In [3]:
p = compute_sha1_padding(b'tt')

<div class="alert alert-block alert-info">

Now, take the SHA-1 secret-prefix MAC of the message you want to forge --- this is just a SHA-1 hash --- and break it into 32 bit SHA-1 registers (SHA-1 calls them "a", "b", "c", &c).

</div>

In [4]:
original_msg = b'This is an authentic message!'
mac = sha1.SHA1(original_msg).finish()

a = mac[0:4]
b = mac[4:8]
c = mac[8:12]
d = mac[12:16]
e = mac[16:20]

<div class="alert alert-block alert-info">

Modify your SHA-1 implementation so that callers can pass in new values for "a", "b", "c" &c (they normally start at magic numbers). With the registers "fixated", hash the additional data you want to forge.

</div>

In [5]:
import struct

def leftrotate(i, n):
    return ((i << n) & 0xffffffff) | (i >> (32 - n))

class evil_SHA1:
    
    def __init__(self, data, sha_state):
        
        a = int.from_bytes(sha_state[0:4], 'big')
        b = int.from_bytes(sha_state[4:8], 'big')
        c = int.from_bytes(sha_state[8:12], 'big')
        d = int.from_bytes(sha_state[12:16], 'big')
        e = int.from_bytes(sha_state[16:20], 'big')
        
        self.h = [a, b, c, d, e]
        self.remainder = data
        self.count = 0

    def _add_chunk(self, chunk):
        self.count += 1
        w = list( struct.unpack(">16I", chunk) + (None,) * (80-16) )
        for i in range(16, 80):
            n = w[i-3] ^ w[i-8] ^ w[i-14] ^ w[i-16]
            w[i] = leftrotate(n, 1)
        a,b,c,d,e = self.h
        for i in range(80):
            f = None
            k = None
            if i < 20:
                f = (b & c) ^ (~b & d)
                k = 0x5A827999
            elif i < 40:
                f = b ^ c ^ d
                k = 0x6ED9EBA1
            elif i < 60:
                f = (b & c) ^ (b & d) ^ (c & d)
                k = 0x8F1BBCDC
            else:
                f = b ^ c ^ d
                k = 0xCA62C1D6

            temp = (leftrotate(a,5) + f + e + k + w[i]) % 2**32
            e = d
            d = c
            c = leftrotate(b, 30)
            b = a
            a = temp
        self.h[0] = (self.h[0] + a) % 2**32
        self.h[1] = (self.h[1] + b) % 2**32
        self.h[2] = (self.h[2] + c) % 2**32
        self.h[3] = (self.h[3] + d) % 2**32
        self.h[4] = (self.h[4] + e) % 2**32

    def add(self, data):
        message = self.remainder + data
        r = len(message) % 64
        if r != 0:
            self.remainder = message[-r:]
        else:
            self.remainder = b''
        for chunk in range(0, len(message)-r, 64):
            self._add_chunk( message[chunk:chunk+64] )
        return self

    def finish(self):
        l = len(self.remainder) + 64 * self.count
        self.add( b'\x80' + b'\x00' * ((55 - l) % 64) + struct.pack(">Q", l * 8) )
        h = tuple(x for x in self.h)
        #self.__init__(self.data, self.sha_state)
        return struct.pack(">5I", *h)

<div class="alert alert-block alert-info">

Using this attack, generate a secret-prefix MAC under a secret key (choose a random word from /usr/share/dict/words or something) of the string:

```"comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon"```

</div>

In [6]:
unknown_MAC_key = Random.get_random_bytes(32)

def make_MAC(message):
    
    my_sha = sha1.SHA1(unknown_MAC_key + bytes(message))
    MAC = my_sha.finish()
    
    return(MAC)

def check_MAC(message, MAC):
    
    return(make_MAC(message) == MAC)
    

In [7]:
original_msg = b'comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon'
original_mac = make_MAC(original_msg)

<div class="alert alert-block alert-info">

Forge a variant of this message that ends with ```";admin=true"```.

</div>

In [8]:
new_msg = b';admin=true'
my_evil_SHA = evil_SHA1(new_msg, original_mac)
forged_mac = my_evil_SHA.finish()

forged_msg = original_msg + compute_sha1_padding(original_msg) + new_msg

print(f"My forged message:\n")
print(forged_msg)
print()
print(f"MAC Check passed?  {check_MAC(forged_msg, forged_mac)}")

My forged message:

b'comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02h;admin=true'

MAC Check passed?  False


---
Well, that first attempt didn't work.  

Thinking it through, I missed a couple of things.  

- The padding includes the length of the message at the end.  For our malicious modified message, that has to include the length of the original message + the length of the new information we're adding.
- The "glue padding" we add has to also account for the length of the key, or it will include the wrong # of 0x00's.  

The original message in this case of ```b'comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon'``` is 77 bytes or 616 bits long.  

The original key was 32 bytes or 256 bits long.

So the total length of the original message was 872 bits (109 bytes).

Padded out -- that would be roundup(872 / 512) = 2 blocks * 512 bits = 1024 bits = 128 bytes in the original message.  

So for the malicious message that adds on to this, we need to account for that 128 bytes.

Here's an updated version of the evil_SHA1 implementation that allows us to specify the length of the authentic message as an additional parameter.

In [9]:
import struct

def leftrotate(i, n):
    return ((i << n) & 0xffffffff) | (i >> (32 - n))

class xtra_evil_SHA1:
    
    def __init__(self, data, sha_state, prev_len):
        
        self.prev_len = prev_len
        
        a = int.from_bytes(sha_state[0:4], 'big')
        b = int.from_bytes(sha_state[4:8], 'big')
        c = int.from_bytes(sha_state[8:12], 'big')
        d = int.from_bytes(sha_state[12:16], 'big')
        e = int.from_bytes(sha_state[16:20], 'big')
        
        self.h = [a, b, c, d, e]
        self.remainder = data
        self.count = 0

    def _add_chunk(self, chunk):
        self.count += 1
        w = list( struct.unpack(">16I", chunk) + (None,) * (80-16) )
        for i in range(16, 80):
            n = w[i-3] ^ w[i-8] ^ w[i-14] ^ w[i-16]
            w[i] = leftrotate(n, 1)
        a,b,c,d,e = self.h
        for i in range(80):
            f = None
            k = None
            if i < 20:
                f = (b & c) ^ (~b & d)
                k = 0x5A827999
            elif i < 40:
                f = b ^ c ^ d
                k = 0x6ED9EBA1
            elif i < 60:
                f = (b & c) ^ (b & d) ^ (c & d)
                k = 0x8F1BBCDC
            else:
                f = b ^ c ^ d
                k = 0xCA62C1D6

            temp = (leftrotate(a,5) + f + e + k + w[i]) % 2**32
            e = d
            d = c
            c = leftrotate(b, 30)
            b = a
            a = temp
            
        self.h[0] = (self.h[0] + a) % 2**32
        self.h[1] = (self.h[1] + b) % 2**32
        self.h[2] = (self.h[2] + c) % 2**32
        self.h[3] = (self.h[3] + d) % 2**32
        self.h[4] = (self.h[4] + e) % 2**32

    def add(self, data):
        message = self.remainder + data
        r = len(message) % 64
        if r != 0:
            self.remainder = message[-r:]
        else:
            self.remainder = b''
        for chunk in range(0, len(message)-r, 64):
            self._add_chunk( message[chunk:chunk+64] )
        return self

    def finish(self):
        l = self.prev_len + len(self.remainder) + 64 * self.count
        self.add( b'\x80' + b'\x00' * ((55 - l) % 64) + struct.pack(">Q", l * 8) )
        h = tuple(x for x in self.h)
        #self.__init__(self.data, self.sha_state)
        return struct.pack(">5I", *h)

Let's try this again...I'll assume we know the length of the key, or have learned it through some easy trial & error.

In [10]:
new_msg = b';admin=true'

FAKE_KEY = b'\x00' * 32
glue_padding = compute_sha1_padding(FAKE_KEY + original_msg)

last_len = (len(FAKE_KEY) + len(original_msg) + len(glue_padding)) 
if (last_len % 64) != 0:
    raise(ValueError('Invalid Length Calculated'))
    
my_evil_SHA = xtra_evil_SHA1(new_msg, original_mac, last_len)

forged_mac = my_evil_SHA.finish()
forged_msg = original_msg + glue_padding + new_msg

print(f"My forged message:\n")
print(forged_msg)
print()
print(f"MAC Check passed?  {check_MAC(forged_msg, forged_mac)}")

My forged message:

b'comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03h;admin=true'

MAC Check passed?  True


[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)